## 1. Importación de librerías

Se importan las librerías necesarias.

In [ ]:
# SOLO EJECUTAR EN ENTORNO GOOGLE COLAB

!pip install plotly_express

In [2]:
import requests
import pandas as pd
import plotly_express as px
import calendar
import time

## 2. Definición de funciones

Se han definido cinco funciones para realizar las labores principales del proyecto.
  - **input_date()** → Asignación del rango de fechas.
  - **extract()** → Petición a la API y recuperación del JSON.
  - **treat_data()** → Tratamiento de los datos en función del tipo.
  - **anual_price()** → Función específica para tratar el formato anual, hace uso de las funciones anteriores.
  - **draw()** → Representación gráfica de los datos.

Cada función tiene su documentación interna y comentarios dentro del código.

In [3]:
def input_date():
    """
    Establece un rango de fechas en formato AAAA-MM-DD

    Retorna:
    --------
    str. Dos variables start_date, end_date.
    """

    start_date = input("Introduce la fecha de inicio en formato AAAA-MM-DD: ")
    end_date = input("Introduce la fecha de fin en formato AAAA-MM-DD: ")
    return start_date, end_date

In [4]:
def extract(url):
    """
    Realiza la petición a la API y recupera el JSON si la conexión es correcta.

    Parámetros:
    -----------
    url: str. Enlace a la API.

    Retorna:
    --------
    dict. Devuelve un diccionario que corresponde al JSON.
    """

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"Se han obtenido los datos correctamente de {url}.")
        return response.json()
    else:
        print("Error al obtener los datos.")

In [5]:
def treat_data(data, type):
    """
    Función encargada de tratar los datos recibidos de la API. Los convierte a un DataFrame de Pandas.

    Parámetros:
    -----------
    data: dict. Datos recibidos de la API.
    type: str. Tipo de datos. Puede ser 'evolucion', 'generacion' o 'intercambio'.

    Retorna:
    --------
    pandas.DataFrame. Devuelve un DataFrame con los datos tratados.
    """

    data_list = []

    if 'included' in data: #Se controla que haya información.

      # En caso de tratarse de una evolución se construye el dataframe de forma simple.
      if type == 'evolucion':
          df = pd.DataFrame(data['included'][0]['attributes']['values'], index=None)

      # Para la energía generada se trata mediante un bucle que va iterando sobre los valores necesarios.
      elif type == 'generacion':
          for i in data['included']:
            title = i['attributes']['title'] # Contiene la tecnología (Eólica, Carbón, Nuclear...)
            energy_type = i['attributes']['type'] # Contiene el tipo de energía (Renovable, No-Renovable)
            for j in i['attributes']['values']: # Se recorren los valores para ese tipo de energía
              values = j['value']
              datetime = j['datetime']
              percentage = j['percentage']
              data_list.append((datetime, title, energy_type, values, percentage)) # Cada tipo de energía se va agregando a la lista auxiliar.

          # Se construye el dataframe con la informacion de la lista auxiliar.
          df = pd.DataFrame(data_list, columns=['datetime', 'tech', 'type', 'value', 'percentage'], index=None)
          df = df[df.tech != "Generación total"] # Se reasigna el df pero sin los valores de "Generación total" en el tipo.

      # Para el intercambio de energía con algún pais se hace algo parecido a lo anterior.
      elif type == 'intercambio':
        for i in data['included']:
          title = i['attributes']['title'] # Contiene la información de exportación e importación.
          for j in i['attributes']['values']:
            values = j['value']
            datetime = j['datetime']
            percentage = j['percentage']
            data_list.append((datetime, title, values, percentage))

        df = pd.DataFrame(data_list, columns=['datetime', 'imp_exp', 'value', 'percentage'], index=None)
        df = df[df.imp_exp != "saldo"] # Se reasigna el df pero sin los valores de "saldo", solo nos interesa exportación e importación.

        # Bucle encargado de poner en positivo los valores de exportanción.
        for i in range(len(df)):
            if df.loc[i, "imp_exp"] == "Exportación":
                df.loc[i, "value"] = abs(df.loc[i, "value"])

      # Se establece la columna 'datetime' como tipo fecha.
      df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
      df['datetime'] = df['datetime'].dt.tz_convert('Europe/Madrid')

      return df

    else:
        print("ERROR. No hay datos en la respuesta.")


- Esta función ha sido necesaria ya que la API solo permite solicitudes mes a mes si se pide que el periodo de tiempo sea por horas.

In [6]:
def anual_price(start_date):
    """
    Función encargada de tratar los datos de forma anual.

    Parámetros:
    -----------
    start_date: str. Fecha de inicio en formato AAAA-MM-DD.

    Retorna:
    --------
    pandas.DataFrame. Devuelve un DataFrame con los datos tratados.
    """

    dfs = []
    year = start_date[:4]
    months = {
        "01": "31",
        "02": "29" if calendar.isleap(int(year)) else "28",  # Controlar bisiesto
        "03": "31",
        "04": "30",
        "05": "31",
        "06": "30",
        "07": "31",
        "08": "31",
        "09": "30",
        "10": "31",
        "11": "30",
        "12": "31"
    }

    # Bucle encargado de realizar las peticiones a la API mes a mes.
    for month, days in months.items():
          start_date = f"{year}-{month}-01"
          end_date = f"{year}-{month}-{days}"

          url = f'https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date={start_date}T00:00&end_date={end_date}T23:59&time_trunc=hour&geo_limit=peninsular&geo_ids=8741'

          data = extract(url)
          df = treat_data(data, 'evolucion')

          dfs.append(df)
          time.sleep(5) # Tiempo de espera NECESARIO. Si se ponen menos segundos se activarán mecanismos de protección de la API.

    return pd.concat(dfs) # Se unen todos los df's de la lista en uno solo.

In [7]:
def draw(df, type, y, x_category=False, color='tech', title='Gráfico', name_x='Fecha', name_y='Valor'):
  """
  Función encargada de representar gráficamente los datos.

  Parámetros:
  -----------
  df: pandas.DataFrame. Datos a representar.
  type: str. Tipo de gráfico. Puede ser 'bar', 'line', 'line_color', 'bar_stack' o 'pie'.
  y: str. Columna a representar
  x_category: bool. Controla si las fechas serán periodos de tiempo o simplemente categorías.
  color: str. Permite establecer las diferentes categorías de los datos a representar.
  title: str. Título del gráfico.
  name_x: str. Nombre del eje x.
  name_y: str. Nombre del eje y.
  """

  # Para cada gráfico se ha establecido que el eje X sea por defecto el tiempo ya que en todos los casos ha sido

  if type == 'bar':
    plot = px.bar(df, x='datetime', y=y, title=title, labels={'datetime':name_x, y:name_y})
    if x_category:
      plot.update_xaxes(type='category')
    plot.show()

  elif type == 'line':
    plot = px.line(df, x='datetime', y=y, title=title, labels={'datetime':name_x, y:name_y})
    plot.show()

  elif type == 'line_color':
    plot = px.line(df, x='datetime', y=y, color=color, title=title, labels={'datetime':name_x, y:name_y})
    plot.update_layout(legend_title='Categorías')
    plot.show()

  elif type == 'bar_stack':
    plot = px.bar(df, x='datetime', y=y, color=color, barmode='stack', title=title, labels={'datetime':name_x, y:name_y})
    plot.update_layout(legend_title='Categorías')
    plot.show()

  elif type == 'pie':
    df_total = df.groupby(color, as_index=False)['percentage'].sum()
    plot = px.pie(df_total, names=color, values=y, color=color, title=title, labels={'datetime':name_x, y:name_y})
    plot.update_layout(legend_title='Categorías')
    plot.update_traces(textinfo='percent+label', hoverinfo='label+percent+value')
    plot.show()

## 3. Representaciones gráficas
A continuación se realizan algunas demostraciones.

In [8]:
start_date, end_date = input_date()

In [9]:
url_demanda = f'https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={start_date}T00:00&end_date={end_date}T23:59&time_trunc=day&geo_limit=peninsular&geo_ids=8741'
url_generacion = f'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={start_date}T00:00&end_date={end_date}T23:59&time_trunc=day&geo_limit=peninsular&geo_ids=8741'
url_francia = f'https://apidatos.ree.es/es/datos/intercambios/francia-frontera?start_date={start_date}T00:00&end_date={end_date}T23:59&time_trunc=day'

In [10]:
demanda = extract(url_demanda)
generacion = extract(url_generacion)
francia = extract(url_francia)

Se han obtenido los datos correctamente de https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2024-01-31T00:00&end_date=2024-12-31T23:59&time_trunc=day&geo_limit=peninsular&geo_ids=8741.
Se han obtenido los datos correctamente de https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2024-01-31T00:00&end_date=2024-12-31T23:59&time_trunc=day&geo_limit=peninsular&geo_ids=8741.
Se han obtenido los datos correctamente de https://apidatos.ree.es/es/datos/intercambios/francia-frontera?start_date=2024-01-31T00:00&end_date=2024-12-31T23:59&time_trunc=day.


In [11]:
df_demanda = treat_data(demanda, 'evolucion')
df_generacion = treat_data(generacion, 'generacion')
df_francia = treat_data(francia, 'intercambio')

In [12]:
df_precio = anual_price(start_date)

Se han obtenido los datos correctamente de https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2024-01-01T00:00&end_date=2024-01-31T23:59&time_trunc=hour&geo_limit=peninsular&geo_ids=8741.
Se han obtenido los datos correctamente de https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2024-02-01T00:00&end_date=2024-02-29T23:59&time_trunc=hour&geo_limit=peninsular&geo_ids=8741.
Se han obtenido los datos correctamente de https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2024-03-01T00:00&end_date=2024-03-31T23:59&time_trunc=hour&geo_limit=peninsular&geo_ids=8741.
Se han obtenido los datos correctamente de https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2024-04-01T00:00&end_date=2024-04-30T23:59&time_trunc=hour&geo_limit=peninsular&geo_ids=8741.
Se han obtenido los datos correctamente de https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date

- Una vez se tienen los datos en las tablas se procede a representar gráficamente.

In [13]:
draw(df_demanda, 'bar', y='value', title=f'Evolución de la demanda energética: de {start_date} a {end_date}', name_y='Demanda')

In [14]:
draw(df_generacion, 'line_color', y='value', title=f'Generación de energía: de {start_date} a {end_date}', name_y='Energía generada MWh')

In [15]:
draw(df_generacion, 'bar_stack', y='value', title=f'Generación de energía: de {start_date} a {end_date}', name_y='Energía generada MWh')

In [16]:
draw(df_generacion, 'pie', y='percentage', title=f'Generación de energía: de {start_date} a {end_date}', name_y='Energía generada MWh')

In [17]:
df_tipo_energia = df_generacion.groupby(['datetime','type'], as_index=False)[['value', 'percentage']].sum()

In [18]:
draw(df_tipo_energia, 'line_color', y='value', color='type', title=f'Evolución tipo de energías: de {start_date} a {end_date}', name_y='Energía generada MWh')

In [19]:
draw(df_tipo_energia, 'bar_stack', y='value', color='type', title=f'Evolución tipo de energías: de {start_date} a {end_date}', name_y='Energía generada MWh')

In [20]:
draw(df_tipo_energia, 'pie', y='percentage', color='type', title=f'Evolución tipo de energías: de {start_date} a {end_date}', name_y='Energía generada MWh')

In [21]:
draw(df_precio, 'line', y='value', color='value', title=f'Precio del Megavatio de {start_date[:4]}', name_y='Precio en €')

In [22]:
precios_caros = df_precio.nlargest(15, 'value').sort_values('value')
precios_baratos = df_precio.nsmallest(15, 'value').sort_values('value')
precios_caros['datetime'] = precios_caros['datetime'].dt.strftime('%Y-%m-%d - %H:%M')
precios_baratos['datetime'] = precios_baratos['datetime'].dt.strftime('%Y-%m-%d - %H:%M')

In [23]:
draw(precios_caros, 'bar', x_category=True, y='value', title=f'15 precios más caros de {start_date[:4]}', name_y='Precio en €')

In [24]:
draw(precios_baratos, 'bar', x_category=True, y='value', title=f'15 precios más baratos de {start_date[:4]}', name_y='Precio en €')

In [28]:
draw(df_francia, 'bar_stack', y='value', color='imp_exp', title=f'Intercambio de energía con Francia: de {start_date} a {end_date}', name_y='Energía intercambiada MWh')

In [29]:
df_francia_pie = df_francia.groupby(['datetime','imp_exp'], as_index=False)[['value', 'percentage']].sum()

In [30]:
draw(df_francia_pie, 'pie', y='percentage', color='imp_exp', title=f'Intercambio de energía con Francia: de {start_date} a {end_date}', name_y='Energía intercambiada MWh')

## 4. Datos de interés
También se pueden extraer algunos datos a través de los dataframes que obtenemos de la API.

In [25]:
df_precio_diario = df_precio.groupby(df_precio['datetime'].dt.date)['value'].mean()

In [26]:
dia_mas_caro = df_precio_diario.idxmax()
precio_mas_caro = df_precio_diario.max()
dia_mas_barato = df_precio_diario.idxmin()
precio_mas_barato = df_precio_diario.min()
precio_medio_anual = df_precio['value'].mean()

In [27]:
print(f"El día más caro de media fue el {dia_mas_caro}, con un precio de {round(precio_mas_caro, 2)}€")
print(f"El día más barato de media fue el {dia_mas_barato}, con un precio de {round(precio_mas_barato, 2)}€")
print(f"El precio medio anual fue de {round(precio_medio_anual, 2)}€")

El día más caro de media fue el 2024-12-12, con un precio de 215.22€
El día más barato de media fue el 2024-03-09, con un precio de 44.45€
El precio medio anual fue de 127.79€
